In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline
import seaborn as sns
sns.set_context("notebook")
import zipfile
from pathlib import Path
plt.style.use('fivethirtyeight')

In [2]:
bus = pd.read_csv(Path("businesses.csv"), encoding="ISO-8859-1")
ins = pd.read_csv(Path("inspections.csv"))
vio = pd.read_csv(Path("violations.csv"))

In [3]:
bus.head()
len(bus["business_id"].unique())

6399

In [35]:
ins.sort_values("date").head()

,business_id,score,date,type,score_level,diffFromPrev,prevScore
5061,11492,88,20150219,routine,3,NaN,NaN
11294,80312,92,20150219,routine,4,NaN,NaN
413,782,93,20150219,routine,4,NaN,NaN
4599,7450,90,20150219,routine,3,NaN,NaN
10513,77755,100,20150219,routine,4,NaN,NaN


In [5]:
vio.head()

,business_id,date,description
0,19,20171211,Inadequate food safety knowledge or lack of ce...
1,19,20171211,Unapproved or unmaintained equipment or utensils
2,19,20160513,Food safety certificate or food handler card n...
3,19,20160513,Unclean or degraded floors walls or ceilings ...
4,19,20160513,Unapproved or unmaintained equipment or utensi...


# Business data

In [6]:
# drop latitude and logitude not complete
#bus.dropna(subset=["latitude","longitude"], inplace=True, axis=0, how="any")

# some missing zip_code
# bus[bus["zip_code"].str[:2]!="94"]

In [7]:
# extract only restaurnats in SF
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]
bus['zip_code'] = bus['postal_code'].str[:5]
bus_sf = bus[bus["zip_code"].isin(validZip)]
bus_sf.head() #3169 rows

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,zip_code
0,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94109
1,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,94104
2,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,94133
3,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051,94110
4,48,ART'S CAFE,747 IRVING ST,San Francisco,CA,94122,37.764013,-122.465749,+14156657440,94122


In [8]:
len(ins[ins["business_id"]==19].index)

2

In [9]:
# put number of inspections into business_table
def calSumIns(bid):
    return len(ins[ins["business_id"]==bid].index)
    
bus_sf["totalIns"]=bus_sf["business_id"].apply(calSumIns)
#bus_sf["numIns2015"]
#bus_sf["numIns2016"]
#bus_sf["numIns2017"]
#bus_sf["numIns2018"]

/Users/weiyilee/miniconda3/envs/data100/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [10]:
bus_sf.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,zip_code,totalIns
0,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94109,2
1,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,94104,3
2,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,94133,1
3,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051,94110,4
4,48,ART'S CAFE,747 IRVING ST,San Francisco,CA,94122,37.764013,-122.465749,+14156657440,94122,2


# Inspection data

In [11]:
# how many unique business_id in inspection: 5758
len(ins["business_id"].unique())

5758

In [12]:
# how many unique business_id in business: 3169
len(bus_sf["business_id"].unique())

6092

In [13]:
# export bus_sf
bus_sf.to_csv("business_sf.csv")

In [14]:
ins.groupby("business_id")

In [15]:
def scoreLevel(data):
    if data<=70:
        return "1"
    elif data<=85:
        return "2"
    elif data<=90:
        return "3"
    else:
        return "4"
ins["score_level"]=ins["score"].apply(scoreLevel)

In [17]:
ins.head()

,business_id,score,date,type,score_level
0,19,94,20171211,routine,4
1,19,94,20160513,routine,4
2,24,96,20160311,routine,4
3,24,98,20161005,routine,4
4,24,98,20171101,routine,4


In [18]:
bus_sf[bus_sf["business_id"]==19]["totalIns"].values[0]

2

In [19]:
numIns2numIDs = ins[["business_id"]].groupby("business_id").size().to_dict()
numIns2numIDs={k:numIns2numIDs[k] for k in numIns2numIDs if numIns2numIDs[k]>1}
len(numIns2numIDs)

4525

In [20]:
def group_to_list(group):
    return list(group)

scores_pairs_by_business = ins[ins["business_id"].isin(numIns2numIDs)][["business_id", "score", "date"]].sort_values("date").groupby("business_id", as_index=False).agg(group_to_list)
scores_pairs_by_business.head()

,business_id,score,date
0,19,"[94, 94]","[20160513, 20171211]"
1,24,"[96, 98, 98]","[20160311, 20161005, 20171101]"
2,45,"[78, 84, 88, 85]","[20160104, 20160614, 20170307, 20170914]"
3,48,"[94, 94]","[20160630, 20180216]"
4,54,"[100, 87]","[20150526, 20170215]"


In [21]:
ins=ins.sort_values(["business_id", "date"]).reset_index(drop=True)
ins.head()

,business_id,score,date,type,score_level
0,19,94,20160513,routine,4
1,19,94,20171211,routine,4
2,24,96,20160311,routine,4
3,24,98,20161005,routine,4
4,24,98,20171101,routine,4


In [27]:
prev_score=-1
prev_id=None
temp={}
temp2={}
for k,v in ins.iterrows():
    if v["business_id"]==prev_id:
        temp[k]=v["score"]-prev_score
        temp2[k]=prev_score
    else:
        prev_id=v["business_id"]
        temp[k]=None
        temp2[k]=None
    prev_score=v["score"]
temp

{0: None,
 1: 0,
 2: None,
 3: 2,
 4: 0,
 5: None,
 6: None,
 7: 6,
 8: 4,
 9: -3,
 10: None,
 11: 0,
 12: None,
 13: -13,
 14: None,
 15: 2,
 16: 2,
 17: None,
 18: -3,
 19: None,
 20: 2,
 21: 4,
 22: -4,
 23: None,
 24: -2,
 25: 2,
 26: -4,
 27: None,
 28: -3,
 29: 7,
 30: -13,
 31: None,
 32: 7,
 33: -4,
 34: None,
 35: -2,
 36: 2,
 37: None,
 38: 5,
 39: -2,
 40: None,
 41: -2,
 42: None,
 43: 2,
 44: None,
 45: 3,
 46: -10,
 47: None,
 48: -2,
 49: 6,
 50: -9,
 51: None,
 52: 2,
 53: -2,
 54: None,
 55: 6,
 56: None,
 57: -8,
 58: 2,
 59: None,
 60: -15,
 61: 7,
 62: None,
 63: None,
 64: 20,
 65: None,
 66: 4,
 67: None,
 68: -2,
 69: None,
 70: -5,
 71: None,
 72: -8,
 73: 9,
 74: None,
 75: 0,
 76: None,
 77: -7,
 78: 0,
 79: None,
 80: 5,
 81: None,
 82: 13,
 83: -21,
 84: None,
 85: -9,
 86: -4,
 87: None,
 88: 10,
 89: -4,
 90: -7,
 91: None,
 92: -8,
 93: None,
 94: 3,
 95: 4,
 96: None,
 97: 8,
 98: 0,
 99: None,
 100: -4,
 101: None,
 102: 4,
 103: None,
 104: 2,
 105: 0,

In [23]:
ins.head()

,business_id,score,date,type,score_level
0,19,94,20160513,routine,4
1,19,94,20171211,routine,4
2,24,96,20160311,routine,4
3,24,98,20161005,routine,4
4,24,98,20171101,routine,4


In [28]:
def rowIndex(row):
    return temp[row.name]
def rowIndex2(row):
    return temp2[row.name]

ins["diffFromPrev"]=ins.apply(rowIndex, axis=1)
ins["prevScore"]=ins.apply(rowIndex2, axis=1)

In [29]:
ins.head()

,business_id,score,date,type,score_level,diffFromPrev,prevScore
0,19,94,20160513,routine,4,NaN,NaN
1,19,94,20171211,routine,4,0.0,94.0
2,24,96,20160311,routine,4,NaN,NaN
3,24,98,20161005,routine,4,2.0,96.0
4,24,98,20171101,routine,4,0.0,98.0


In [30]:
# export ins_sf
ins.to_csv("inspection_sf.csv")

In [32]:
ins[ins["diffFromPrev"]<0]

,business_id,score,date,type,score_level,diffFromPrev,prevScore
9,45,85,20170914,routine,2,-3.0,88.0
13,54,87,20170215,routine,3,-13.0,100.0
18,58,70,20170918,routine,1,-3.0,73.0
22,61,94,20171128,routine,4,-4.0,98.0
24,66,98,20160322,routine,4,-2.0,100.0
26,66,96,20170703,routine,4,-4.0,100.0
28,67,87,20160401,routine,3,-3.0,90.0
30,67,81,20170804,routine,2,-13.0,94.0
33,73,88,20171129,routine,3,-4.0,92.0
35,76,98,20161005,routine,4,-2.0,100.0


In [37]:
bus_sf[bus_sf["totalIns"]>1].shape

(4378, 11)

In [38]:
bus_sf.shape

(6092, 11)

In [39]:
4378/6092

0.7186474064346684

In [40]:
bus_sf.shape

(6092, 11)